In [ ]:
# To get run time for each cell
!pip install ipython-autotime -q
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.21 s (started: 2024-09-27 10:11:00 +00:00)


In [ ]:
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py310hde88566_4.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.10/dist-packages/ lib/python3.10/site-packages/talib --strip-components=3

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:02:01 --:--:--     0^C
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:06 --:--:--     0^C
time: 2min 8s (started: 2024-09-27 10:11:03 +00:00)


In [ ]:
# Basic Libraries
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

# Pytorch Library
import torch
import torch.nn as nn
import torch.optim as optim
import torch.cuda as cuda
import talib
import math

# Utility Libraries
from tqdm import tqdm
import os
import warnings
warnings.filterwarnings("ignore")

# Set seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)


time: 2.28 ms (started: 2024-09-27 10:18:25 +00:00)


In [ ]:
# Function to calculate technical indicators
def calculate_technical_indicators(df, normalise=False):

    # Normalise the data if needed
    if normalise:
        x = df.values
        standard_scaler = StandardScaler()
        x_scaled = standard_scaler.fit_transform(x)
        df = pd.DataFrame(x_scaled, columns=df.columns)

    # Calculate technical indicators using TA-Lib
    df['RSI'] = talib.RSI(df['close'], 20)

    upper, middle, lower = talib.BBANDS(df['close'], timeperiod=20, nbdevup=2, nbdevdn=2)
    df['BBANDS_Upper'] = upper
    df['BBANDS_Middle'] = middle
    df['BBANDS_Lower'] = lower

    macd, signal, hist = talib.MACD(df['close'], fastperiod=3, slowperiod=18, signalperiod=6)
    df['MACD'] = macd
    df['SIGNAL'] = signal
    df['HIST'] = hist

    df['OBV'] = talib.OBV(df['close'], df['volume'])
    df['EMA'] = talib.EMA(df['close'], timeperiod=20)

    df['ATR'] = talib.ATR(df['high'], df['low'], df['close'], timeperiod=14)
    df['CCI'] = talib.CCI(df['high'], df['low'], df['close'], timeperiod=14)
    df['ROC'] = talib.ROC(df['close'], timeperiod=10)
    df['WILLR'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=14)

    df = df[23:].reset_index(drop=True)
    return df

time: 1.4 ms (started: 2024-09-27 10:18:26 +00:00)


In [ ]:
# Function to prepare training and testing data
def prepare_train_test_data(df, days_input, days_output, train_split, normalise=False, shuffle=False):

    # Compute the difference between consecutive data points
    # df = df.diff().dropna().reset_index(drop=True)

    # Normalise the data if needed
    if normalise:
        x = df.values
        standard_scaler = StandardScaler()
        x_scaled = standard_scaler.fit_transform(x)
        df = pd.DataFrame(x_scaled, columns=df.columns)

    # Split data into training and testing
    train_indices = int(df.shape[0] * train_split)
    train_data = df[:train_indices]
    test_data = df[train_indices:]
    test_data = test_data.reset_index(drop=True)

    # Get the train and test Y
    train_y = df['close'][:train_indices]
    test_y = df['close'][train_indices:]
    test_y = test_y.reset_index(drop=True)

    train_sequences_X = [train_data[i: i + days_input].values for i in range(0, len(train_data) - (days_input + days_output - 1))]
    train_sequences_y = [train_y[i: i + days_output] for i in range(days_input, len(train_y) - (days_output - 1))]
    test_sequences_X = [test_data[i: i + days_input].values for i in range(0, len(test_data) - (days_input + days_output - 1))]
    test_sequences_y = [test_y[i: i + days_output] for i in range(days_input, len(test_y) - (days_output - 1))]

    # Split data into sequences
    if not shuffle:
        train_sequences_X = np.array(train_sequences_X)
        train_sequences_y = np.array(train_sequences_y)
        test_sequences_X = np.array(test_sequences_X)
        test_sequences_y = np.array(test_sequences_y)

        return train_sequences_X, train_sequences_y, test_sequences_X, test_sequences_y, test_data

    # Shuffle data
    indices = np.random.permutation(len(train_sequences_X))
    X_train = np.array([train_sequences_X[i] for i in indices])
    y_train = np.array([train_sequences_y[i] for i in indices])

    return X_train, y_train, np.array(test_sequences_X), np.array(test_sequences_y), test_data

time: 1.66 ms (started: 2024-09-27 10:18:26 +00:00)


In [ ]:
# Replace the csv_files list with this function
def get_csv_files(folder_path):
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    return [os.path.join(folder_path, f) for f in csv_files]

# Function to assign trend signs to an array
# def assign_trend_sign(arr):
#     trend_signs = np.sign(arr)  # Get signs of all values
#     trend_signs[trend_signs > 0] = 1  # Map positive values to 1
#     trend_signs[trend_signs < 0] = -1  # Map negative values to -1
#     return trend_signs

def assign_trend_sign(arr):
    trend_signs = np.zeros_like(arr)  # Initialize an array of zeros with the same shape as arr
    trend_signs[1:] = np.sign(arr[1:] - arr[:-1])  # Compare each element with the previous one
    return trend_signs

# Function to count matching elements in two arrays
def count_matching_elements(arr1, arr2):
    if arr1.shape != arr2.shape:
        raise ValueError("Input arrays must have the same shape")

    matching_count = np.sum(arr1 == arr2)
    total_elements = np.prod(arr1.shape)  # Total number of elements in the array
    matching_percentage = (matching_count / total_elements) * 100  # Calculate percentage

    return matching_percentage

def calculate_sharpe_ratio(returns, risk_free_rate=0.02):
    excess_returns = returns - risk_free_rate
    return np.sqrt(252) * excess_returns.mean() / excess_returns.std()


time: 1.98 ms (started: 2024-09-27 10:18:26 +00:00)


In [ ]:
import math
import torch.nn as nn

class PositionalEncodingCNN(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncodingCNN, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        self.d_model = d_model

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        # Expand the positional encoding to match the input shape
        batch_size, features, seq_len = x.size()
        pe = self.pe
        pe = pe.permute(2, 1, 0)
        pe = pe[:,:,:seq_len]
        pe_1 = pe.repeat(1, features +1, 1)
        # print(f"X shape: {x.shape}")
        # print(f"PE shape: {pe.shape}")
        x = torch.cat((x, pe), dim=1)
        x = x + pe_1
        return self.dropout(x)



time: 870 µs (started: 2024-09-27 10:18:26 +00:00)


In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, kernel_sizes, batch_size, stride=1, dilation=1, activation=nn.ReLU(), num_heads=8, dropout_rate=0.2):
        super(ConvBlock, self).__init__()

        # Convolution layers
        self.conv_layers = nn.ModuleList([
            nn.Conv1d(in_channels, hidden_channels, kernel_size, stride=stride, padding="same", dilation=dilation)
            for kernel_size in kernel_sizes
        ])

        # Batch layer for Convolution Layer
        self.bn_layers = nn.ModuleList([nn.BatchNorm1d(hidden_channels) for _ in kernel_sizes])

        # Positional Encoder
        self.pos_encoder = PositionalEncodingCNN(batch_size, dropout_rate)

        # MLP + Batch Normal layer
        self.mlp = nn.Conv1d((hidden_channels * len(kernel_sizes)) + 1, out_channels, kernel_size=1)
        self.bn_final = nn.BatchNorm1d(out_channels)

        # Activation Layer
        self.activation = activation

    def forward(self, input):
        # Transposing the input to match the time dimension
        input = input.transpose(1, 2)

        # Taking all of the convolutions and concatenating them
        conv_outputs = [bn(conv(input)) for conv, bn in zip(self.conv_layers, self.bn_layers)]
        total_conv = torch.cat(conv_outputs, dim=1)

        # print(f"PE input{total_conv.shape}")

        # Apply Positional Encoder
        total_conv = self.pos_encoder(total_conv)
        # print(f"PE output{total_conv.shape}")

        # Applying the final convolution to change the number of features
        output = self.mlp(total_conv)

        output = self.bn_final(output)

        # Activation
        # output = self.activation(output)

        output = output.transpose(1, 2)

        return output

time: 808 µs (started: 2024-09-27 10:18:26 +00:00)


In [ ]:
# import torch
# import torch.nn as nn

# class TransformerWithAttention(nn.Module):
#     def __init__(self, input_size, hidden_size, hidden_size2, seq_len, num_layers, num_heads, dropout_rate=0.3):
#         super(TransformerWithAttention, self).__init__()

#         self.input_size = input_size
#         self.hidden_size = hidden_size
#         self.seq_len = seq_len
#         self.num_layers = num_layers
#         self.num_heads = num_heads

#         # Input embedding
#         self.input_embedding = nn.Linear(input_size, hidden_size)

#         # Positional encoding
#         self.positional_encoding = PositionalEncoding(hidden_size, dropout_rate, seq_len)

#         # Transformer Encoder 1
#         encoder_layer1 = nn.TransformerEncoderLayer(d_model=hidden_size,
#                                                     nhead=num_heads,
#                                                     dim_feedforward=hidden_size * 4,
#                                                     dropout=dropout_rate,
#                                                     activation='gelu',
#                                                     batch_first=True)
#         self.transformer_encoder1 = nn.TransformerEncoder(encoder_layer1, num_layers=num_layers)

#         # Transformer Encoder 2
#         encoder_layer2 = nn.TransformerEncoderLayer(d_model=hidden_size,
#                                                     nhead=num_heads,
#                                                     dim_feedforward=hidden_size * 4,
#                                                     dropout=dropout_rate,
#                                                     activation='gelu',
#                                                     batch_first=True)
#         self.transformer_encoder2 = nn.TransformerEncoder(encoder_layer2, num_layers=num_layers)

#         # Transformer Encoder 3
#         encoder_layer3 = nn.TransformerEncoderLayer(d_model=hidden_size,
#                                                     nhead=num_heads,
#                                                     dim_feedforward=hidden_size * 4,
#                                                     dropout=dropout_rate,
#                                                     activation='gelu',
#                                                     batch_first=True)
#         self.transformer_encoder3 = nn.TransformerEncoder(encoder_layer3, num_layers=num_layers)

#         # Multi-Head Attention between Encoder and Decoder layers
#         self.multihead_attention1 = nn.MultiheadAttention(embed_dim=hidden_size, num_heads=num_heads, batch_first=True)
#         self.multihead_attention2 = nn.MultiheadAttention(embed_dim=hidden_size, num_heads=num_heads, batch_first=True)
#         self.multihead_attention3 = nn.MultiheadAttention(embed_dim=hidden_size, num_heads=num_heads, batch_first=True)

#         # Transformer Decoder 1
#         decoder_layer1 = nn.TransformerDecoderLayer(d_model=hidden_size,
#                                                     nhead=num_heads,
#                                                     dim_feedforward=hidden_size * 4,
#                                                     dropout=dropout_rate,
#                                                     activation='gelu',
#                                                     batch_first=True)
#         self.transformer_decoder1 = nn.TransformerDecoder(decoder_layer1, num_layers=num_layers)

#         # Transformer Decoder 2
#         decoder_layer2 = nn.TransformerDecoderLayer(d_model=hidden_size,
#                                                     nhead=num_heads,
#                                                     dim_feedforward=hidden_size * 4,
#                                                     dropout=dropout_rate,
#                                                     activation='gelu',
#                                                     batch_first=True)
#         self.transformer_decoder2 = nn.TransformerDecoder(decoder_layer2, num_layers=num_layers)

#         # Transformer Decoder 3
#         decoder_layer3 = nn.TransformerDecoderLayer(d_model=hidden_size,
#                                                     nhead=num_heads,
#                                                     dim_feedforward=hidden_size * 4,
#                                                     dropout=dropout_rate,
#                                                     activation='gelu',
#                                                     batch_first=True)
#         self.transformer_decoder3 = nn.TransformerDecoder(decoder_layer3, num_layers=num_layers)

#         # Output layer
#         self.output_layer = nn.Linear(hidden_size, input_size)

#         # Normalization layers
#         self.norm_hidden = nn.LayerNorm(hidden_size)
#         self.norm_output = nn.LayerNorm(input_size)

#         # GELU and dropout layers
#         self.gelu = nn.GELU()
#         self.dropout = nn.Dropout(dropout_rate)

#     def forward(self, input):
#         # Input embedding
#         embedded_input = self.input_embedding(input)

#         # Positional encoding
#         encoded_input = self.positional_encoding(embedded_input)

#         # Transformer Encoder 1
#         encoder_output1 = self.transformer_encoder1(encoded_input)
#         encoder_output1 = self.norm_hidden(self.gelu(encoder_output1))

#         # Multi-Head Attention 1
#         attn_output1, _ = self.multihead_attention1(encoder_output1, encoder_output1, encoder_output1)
#         attn_output1 = self.norm_hidden(self.gelu(attn_output1))

#         # Transformer Encoder 2
#         encoder_output2 = self.transformer_encoder2(attn_output1)
#         encoder_output2 = self.norm_hidden(self.gelu(encoder_output2))

#         # Multi-Head Attention 2
#         attn_output2, _ = self.multihead_attention2(encoder_output2, encoder_output2, encoder_output2)
#         attn_output2 = self.norm_hidden(self.gelu(attn_output2))

#         # Transformer Encoder 3
#         encoder_output3 = self.transformer_encoder3(attn_output2)
#         encoder_output3 = self.norm_hidden(self.gelu(encoder_output3))

#         # Multi-Head Attention 3
#         attn_output3, _ = self.multihead_attention3(encoder_output3, encoder_output3, encoder_output3)
#         attn_output3 = self.norm_hidden(self.gelu(attn_output3))

#         # Transformer Decoder 1
#         decoder_output1 = self.transformer_decoder1(attn_output3, attn_output3)
#         decoder_output1 = self.norm_hidden(self.gelu(decoder_output1))

#         # Transformer Decoder 2
#         decoder_output2 = self.transformer_decoder2(decoder_output1, decoder_output1)
#         decoder_output2 = self.norm_hidden(self.gelu(decoder_output2))

#         # Transformer Decoder 3
#         decoder_output3 = self.transformer_decoder3(decoder_output2, decoder_output2)
#         decoder_output3 = self.norm_hidden(self.gelu(decoder_output3))

#         # Output processing
#         output = self.output_layer(decoder_output3)
#         output = self.norm_output(output)

#         # Return only the last time step prediction
#         return output[:, -1, :]

# # Positional Encoding Class remains unchanged
# class PositionalEncoding(nn.Module):
#     def __init__(self, d_model, dropout=0.1, max_len=5000):
#         super(PositionalEncoding, self).__init__()
#         self.dropout = nn.Dropout(p=dropout)

#         pe = torch.zeros(max_len, d_model)
#         position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
#         div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
#         pe[:, 0::2] = torch.sin(position * div_term)
#         pe[:, 1::2] = torch.cos(position * div_term)
#         pe = pe.unsqueeze(0).transpose(0, 1)
#         self.register_buffer('pe', pe)

#     def forward(self, x):
#         x = x + self.pe[:x.size(0), :]
#         return self.dropout(x)


time: 249 µs (started: 2024-09-27 10:18:26 +00:00)


In [ ]:
import torch
import torch.nn as nn
import math

class SelfAttention(nn.Module):
    def __init__(self, hidden_size, num_heads, dropout_rate=0.1):
        super().__init__()
        self.mha = nn.MultiheadAttention(hidden_size, num_heads, dropout=dropout_rate, batch_first=True)

    def forward(self, x):
        return self.mha(x, x, x, need_weights=False)[0]

class CrossAttention(nn.Module):
    def __init__(self, hidden_size, num_heads, dropout_rate=0.1):
        super().__init__()
        self.mha = nn.MultiheadAttention(hidden_size, num_heads, dropout=dropout_rate, batch_first=True)

    def forward(self, x, context):
        return self.mha(x, x, context, need_weights=False)[0]

class TransformerWithAttention(nn.Module):
    def __init__(self, input_size, hidden_size, hidden_size2, seq_len, num_layers, num_heads, dropout_rate=0.3):
        super(TransformerWithAttention, self).__init__()

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_len = seq_len
        self.num_layers = num_layers
        self.num_heads = num_heads

        # Input embedding
        self.input_embedding = nn.Linear(input_size, hidden_size)

        # Positional encoding
        self.positional_encoding = PositionalEncoding(hidden_size, dropout_rate, seq_len)

        # Transformer Encoders and Self-Attention
        self.transformer_encoders = nn.ModuleList([
            nn.TransformerEncoderLayer(d_model=hidden_size,
                                       nhead=num_heads,
                                       dim_feedforward=hidden_size * 4,
                                       dropout=dropout_rate,
                                       activation='gelu',
                                       batch_first=True)
            for _ in range(3)
        ])
        self.self_attentions = nn.ModuleList([
            SelfAttention(hidden_size, num_heads, dropout_rate)
            for _ in range(2)  # One less than encoder layers
        ])

        # Multi-Head Attention layers
        self.multihead_attentions = nn.ModuleList([
            nn.MultiheadAttention(embed_dim=hidden_size, num_heads=num_heads, batch_first=True)
            for _ in range(5)
        ])

        # Transformer Decoders and Cross-Attention
        self.transformer_decoders = nn.ModuleList([
            nn.TransformerDecoderLayer(d_model=hidden_size,
                                       nhead=num_heads,
                                       dim_feedforward=hidden_size * 4,
                                       dropout=dropout_rate,
                                       activation='gelu',
                                       batch_first=True)
            for _ in range(3)
        ])
        self.cross_attentions = nn.ModuleList([
            CrossAttention(hidden_size, num_heads, dropout_rate)
            for _ in range(2)  # One less than decoder layers
        ])

        # Output layer
        self.output_layer = nn.Linear(hidden_size, input_size)

        # Normalization layers
        self.norm_hidden = nn.LayerNorm(hidden_size)
        self.norm_output = nn.LayerNorm(input_size)

        # Dropout layer
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, input):
        # Input embedding
        x = self.input_embedding(input)

        # Positional encoding
        layer_0 = self.positional_encoding(x)

        # Transformer Encoder 1
        layer_1 = self.transformer_encoders[0](layer_0)
        layer_1 = self.norm_hidden(layer_1)

        # Self-Attention 1
        self_attn_1 = self.self_attentions[0](layer_1)
        self_attn_1 = self.norm_hidden(self_attn_1)

        # Transformer Encoder 2
        layer_2 = self.transformer_encoders[1](self_attn_1)
        layer_2 = self.norm_hidden(layer_2)

        # Self-Attention 2
        self_attn_2 = self.self_attentions[1](layer_2)
        self_attn_2 = self.norm_hidden(self_attn_2)

        # Transformer Encoder 3
        layer_3 = self.transformer_encoders[2](self_attn_2)
        layer_3 = self.norm_hidden(layer_3)

        # Attention 3
        attention_3, _ = self.multihead_attentions[2](layer_3, layer_3, layer_3)
        attention_3 = self.norm_hidden(attention_3)

        # Transformer Decoder 1
        layer_3 = self.transformer_decoders[0](attention_3, attention_3)
        layer_3 = self.norm_hidden(layer_3)

        # Cross-Attention 1
        cross_attn_1 = self.cross_attentions[0](layer_3, self_attn_2)
        cross_attn_1 = self.norm_hidden(cross_attn_1)

        # Transformer Decoder 2
        layer_2 = self.transformer_decoders[1](cross_attn_1, cross_attn_1)
        layer_2 = self.norm_hidden(layer_2)

        # Cross-Attention 2
        cross_attn_2 = self.cross_attentions[1](layer_2, self_attn_1)
        cross_attn_2 = self.norm_hidden(cross_attn_2)

        # Transformer Decoder 3
        layer_1 = self.transformer_decoders[2](cross_attn_2, cross_attn_2)
        layer_1 = self.norm_hidden(layer_1)

        # Output processing
        output = self.output_layer(layer_1)
        output = self.norm_output(output)

        # Return only the last time step prediction
        return output[:, -1, :]

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

time: 2.48 ms (started: 2024-09-27 10:18:26 +00:00)


In [ ]:
class CNNLSTMWithSelfAttentionLNN(nn.Module):
    def __init__(self, input_size, hidden_size_1, hidden_size_2, hidden_size_3, hidden_size_4, lstm_layers, output_size, kernel_sizes, seq_len, num_days, batch_size, device, l2_lambda, dropout_rate=0.4, num_heads=16):
        super(CNNLSTMWithSelfAttentionLNN, self).__init__()

        # Backbone attribues
        self.input_size = input_size
        self.kernel_sizes = kernel_sizes
        self.hidden_size_1 = hidden_size_1
        self.hidden_size_2 = hidden_size_2
        self.hidden_size_3 = hidden_size_3
        self.hidden_size_4 = hidden_size_4
        self.lstm_layers = lstm_layers
        self.num_heads = num_heads
        self.output_size = output_size

        # Stock Data Attributes
        self.seq_len = seq_len
        self.num_days = num_days

        # Dropout rate
        self.dropout_rate = dropout_rate
        self.l2_lambda = l2_lambda
        self.batch_size = batch_size
        self.device = device

        # Hidden states
        self.h = None
        self.c = None

        # Backbone models
        self.conv_blocks = ConvBlock(self.input_size, self.hidden_size_1, self.hidden_size_2, self.kernel_sizes, self.batch_size, dropout_rate=self.dropout_rate)
        # self.lstm_block = LSTMWithAttention(self.hidden_size_2, self.hidden_size_3, self.hidden_size_4, self.seq_len, self.lstm_layers, self.num_heads, dropout_rate=self.dropout_rate)
        self.lstm_block = TransformerWithAttention(self.hidden_size_2, self.hidden_size_3, self.hidden_size_4, self.seq_len, self.lstm_layers, self.num_heads, dropout_rate=self.dropout_rate)

        # Linear Model, RELU and dropout
        self.fc = nn.Linear(hidden_size_2, output_size)
        self.dropout = nn.Dropout(dropout_rate)
        self.relu = nn.ReLU()


    def init_hidden(self, batch_size):
        self.h = torch.zeros(self.lstm_layers, batch_size, self.hidden_size_3).to(self.device)
        self.c = torch.zeros(self.lstm_layers, batch_size, self.hidden_size_3).to(self.device)

    def reset_hidden(self):
        self.h = None
        self.c = None

    def forward(self, x):

        # print("\n******************** CONVOLUTION BLOCK ******************** \n\n")

        # Process input through the convolutional block
        conv_output = self.conv_blocks(x)
        # print(f"Convolution output shape: {conv_output.shape}")
        # print(f"Convolution output: {conv_output}")

        # print("******************** LSTM BLOCK ******************** \n\n")

        batch_size = x.size(0)

        # Initialize hidden states if not already done
        if self.h is None or self.c is None or batch_size != self.h.size(1):
            self.init_hidden(batch_size)

        # Initialize the hidden state
        h = self.h.detach()
        c = self.c.detach()

        # Initialize the prediction_tensor
        predictions_tensor = torch.tensor([]).to(x.device)

        # Loop through the sequence length
        for i in range(self.num_days):

          # print(f"******************** Day {i+1} ******************** \n\n")


          # Process input through the LSTM with attention block
          # lstm_output, h, c = self.lstm_block(conv_output[:, -self.seq_len:, :], h, c)
          lstm_output= self.lstm_block(conv_output[:, -self.seq_len:, :])
          # print(f"LSTM output shape: {lstm_output.shape}")
          # print(f"LSTM output: {lstm_output}")


          linear_output = self.fc(lstm_output)
          # print(f"Linear output shape: {linear_output.shape}")
          # print(f"Linear output: {linear_output}")

          predictions_tensor = torch.cat((predictions_tensor, linear_output), dim=1)
          # print(f"Predictions tensor shape for day {i+1}: {predictions_tensor.shape}")

          conv_output = torch.cat((conv_output, lstm_output.unsqueeze(1)), dim=1)
          # print(f"Conv output shape for day {i+1}: {conv_output.shape}")

        # print(f"Final predictions tensor shape: {predictions_tensor.shape}")

        self.h = h.detach()
        self.c = c.detach()

        return predictions_tensor[:, -1:]

    def loss(self, outputs, labels):
        # print(f"Outputs shape: {outputs.shape}")
        # print(f"Labels shape: {labels.shape}")

        # L2 regularisation
        # l2_norm = sum(p.pow(2.0).sum() for p in self.parameters())
        loss = nn.MSELoss()(outputs, labels )

        return loss

time: 1.24 ms (started: 2024-09-27 10:18:26 +00:00)


In [ ]:
# Function to train the model
def train(model, train_dataset, val_dataset, optimizer, scheduler, epochs, batch_size, device, save_path=None):
    model.to(device)

    # Initialize the training and validation dataset
    X_train, y_train = train_dataset
    X_val, y_val = val_dataset
    train_loss_total = []
    val_loss_total = []
    matching_elements = []
    matching_elements_shifted = []

    # Initialize the best_loss and num_batches
    best_val_loss = float('inf')
    num_batches = len(X_train) // batch_size

    # Loop for traning rounds
    for epoch in range(epochs):

        # # Reset hidden states at the beginning of each batch
        # model.reset_hidden()

        model.train()
        total_loss = 0
        progress_bar = tqdm(range(num_batches), desc=f'Epoch {epoch+1}', unit='batch')

        for batch_idx in progress_bar:

            # Getting the start and end index
            start_idx = batch_idx * batch_size
            end_idx = start_idx + batch_size

            # Getting the training batch
            inputs = X_train[start_idx:end_idx, :, :].float().to(device)
            labels = y_train[start_idx:end_idx, :].float().to(device)
            # print(f"Inputs shape: {inputs.shape}")
            # print(f"Labels shape: {labels.shape})

            # Traning the model for the epoch
            outputs = model(inputs)
            # print(f"Output shape right out of the model: {outputs.shape}")
            loss = model.loss(outputs, labels)

            # Backpropogation and updation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()


            avg_loss = total_loss / (batch_idx + 1)
            progress_bar.set_postfix(loss=avg_loss)

        print(f"Validating...")

        # Validate the model
        val_loss = validate(model, val_dataset, batch_size, device)

        # Changing the learning rate according to the loss
        scheduler.step(val_loss)

        print(f'Epoch {epoch+1}, Training Loss: {total_loss / num_batches}, Validation Loss: {val_loss}\n')
        train_loss_total.append(total_loss / num_batches)
        val_loss_total.append(val_loss)

        # Calculate matching elements
        _, outputs, labels = test(model, val_dataset, batch_size, device)
        outputs_np = outputs.cpu().numpy().flatten()
        labels_np = labels.cpu().numpy().flatten()

        array_output = assign_trend_sign(outputs_np)
        array_labels = assign_trend_sign(labels_np)
        matching = count_matching_elements(array_output, array_labels)
        matching_elements.append(matching)

        array_output_shifted = assign_trend_sign(np.roll(outputs_np, -1))
        matching_shifted = count_matching_elements(array_output_shifted, array_labels)
        matching_elements_shifted.append(matching_shifted)

        # Save the model weights if the validation loss is the best we've seen so far.
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            if save_path:
                torch.save(model.state_dict(), save_path)
                print(f'Best model saved with validation loss: {best_val_loss:.4f}\n')

    return np.array(train_loss_total), np.array(val_loss_total), np.array(matching_elements), np.array(matching_elements_shifted)

time: 1.17 ms (started: 2024-09-27 10:18:26 +00:00)


In [ ]:
# Function to validate the model
def validate(model, dataset, batch_size, device):

    model.eval()
    X_val, y_val = dataset

    num_batches = len(X_val) // batch_size
    total_loss = 0

    with torch.no_grad():
        for batch_idx in range(num_batches):


            # Getting the start and end index
            start_idx = batch_idx * batch_size
            end_idx = start_idx + batch_size

            # Getting the validation batch
            inputs = X_val[start_idx:end_idx, :, :].float().to(device)
            labels = y_val[start_idx:end_idx, :].float().to(device)

            outputs = model(inputs)
            # print(outputs)
            loss = model.loss(outputs, labels)
            total_loss += loss.item()

    avg_loss = total_loss / num_batches
    return avg_loss

time: 530 µs (started: 2024-09-27 10:18:26 +00:00)


In [ ]:
# Function to test the model
def test(model, dataset, batch_size, device):

    model.to(device)
    model.eval()
    X_test, y_test = dataset

    num_batches = len(X_test) // batch_size
    total_loss = 0

    with torch.no_grad():
      for batch_idx in range(num_batches):


            # Getting the start and end index
            start_idx = batch_idx * batch_size
            end_idx = start_idx + batch_size

            # Getting the validation batch
            inputs = X_test[start_idx:end_idx, :, :].float().to(device)
            labels = y_test[start_idx:end_idx, :].float().to(device)

            outputs = model(inputs)
            # print(f"Outputs shape: {outputs.shape}")
            # print(f"Labels shape: {labels.shape}")
            loss = model.loss(outputs, labels)
            total_loss += loss.item()

    avg_loss = total_loss / num_batches
    print(f'Test Loss: {avg_loss}')
    return avg_loss, outputs, labels

time: 487 µs (started: 2024-09-27 10:18:26 +00:00)


In [ ]:
# Modify the process_all_files function
def process_all_files(folder_path):
    results = {}
    csv_files = get_csv_files(folder_path)

    for file in csv_files:
        print(f"*********************************     Processing {file}...     *********************************")

        # Load and preprocess data
        stock_df = pd.read_csv(file)
        stock_df = stock_df[['high', 'low', 'open', 'close', 'volume']]
        stock_df['volume'] = stock_df['volume'].astype('float64')
        stock_df = calculate_technical_indicators(stock_df, normalise=False)

        # Prepare data
        X_train, y_train, X_test, y_test, test_data = prepare_train_test_data(stock_df, days_input, days_output, train_split, normalise=True, shuffle=True)

        # Convert to PyTorch tensors
        X_train = torch.from_numpy(X_train).double()
        y_train = torch.from_numpy(y_train).double()
        X_test = torch.from_numpy(X_test).double()
        y_test = torch.from_numpy(y_test).double()

        # Prepare datasets
        train_size = int(training_size * len(X_train))
        train_dataset = (X_train[:train_size, :, :], y_train[:train_size, :])
        val_dataset = (X_train[train_size:, :, :], y_train[train_size:, :])

        # Initialize model and optimizer
        model = CNNLSTMWithSelfAttentionLNN(input_size, hidden_conv_size, hidden_mlp_size, hidden_lstm_size_1, hidden_lstm_size_2, lstm_layers, output_size, kernel_sizes, seq_len, num_days, batch_size, device, l2_lambda, dropout_rate, num_heads)
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, verbose=True)

        # Create a unique save path for each model
        # model_save_path = f'model_{os.path.basename(file)}.pth'
        model_save_path = f'model.pth'

        # Load existing model if available
        if os.path.exists(model_save_path):
            print(f"Loading existing model for {file}...")
            model.load_state_dict(torch.load(model_save_path))
        else:
            print(f"Training new model for {file}...")

        # Train model
        train_loss, val_loss, matching, matching_shifted = train(model, train_dataset, val_dataset, optimizer, scheduler, epochs=epochs, batch_size=batch_size, device=device, save_path=model_save_path)

        results[os.path.basename(file)] = {
            'train_loss': train_loss,
            'val_loss': val_loss,
            'matching': matching,
            'matching_shifted': matching_shifted
        }

    return results

time: 949 µs (started: 2024-09-27 10:18:26 +00:00)


In [ ]:
def plot_results(results, epochs):
    num_files = len(results)

    # Adjust vertical and horizontal spacing
    vertical_spacing = max(0.03, min(0.1, 0.3 / num_files))
    horizontal_spacing = 0.02

    # Create subplots with more descriptive titles
    fig = make_subplots(
        rows=num_files, cols=4,
        subplot_titles=[
            item for file in results.keys() for item in
            [f"{file} - Train Loss", f"{file} - Val Loss",
             f"{file} - Matching %", f"{file} - Matching Shifted %"]
        ],
        vertical_spacing=vertical_spacing,
        horizontal_spacing=horizontal_spacing
    )

    for row, (file, data) in enumerate(results.items(), start=1):
        # Training Loss
        fig.add_trace(go.Scatter(x=list(range(1, epochs+1)), y=data['train_loss'],
                                 name=f'{file} - Train', mode='lines+markers'), row=row, col=1)

        # Validation Loss
        fig.add_trace(go.Scatter(x=list(range(1, epochs+1)), y=data['val_loss'],
                                 name=f'{file} - Val', mode='lines+markers'), row=row, col=2)

        # Matching Elements
        fig.add_trace(go.Scatter(x=list(range(1, epochs+1)), y=data['matching'],
                                 name=f'{file} - Match', mode='lines+markers'), row=row, col=3)

        # Matching Elements (Shifted)
        fig.add_trace(go.Scatter(x=list(range(1, epochs+1)), y=data['matching_shifted'],
                                 name=f'{file} - Match Shifted', mode='lines+markers'), row=row, col=4)

    # Update layout with increased height per subplot and adjusted width
    fig.update_layout(
        height=250*num_files,  # Increase height per subplot
        width=1800,            # Slightly increase overall width
        title_text="Model Performance Across Different Stocks",
        showlegend=False,
        title_x=0.5,           # Center the main title
    )

    # Update y-axes titles and range
    for i in range(1, num_files+1):
        fig.update_yaxes(title_text="Loss", row=i, col=1, range=[0, max(results[list(results.keys())[i-1]]['train_loss'])*1.1])
        fig.update_yaxes(title_text="Loss", row=i, col=2, range=[0, max(results[list(results.keys())[i-1]]['val_loss'])*1.1])
        fig.update_yaxes(title_text="Matching %", row=i, col=3, range=[0, 100])
        fig.update_yaxes(title_text="Matching %", row=i, col=4, range=[0, 100])

    # Update x-axes titles
    for i in range(1, num_files*4+1):
        fig.update_xaxes(title_text="Epochs", row=(i-1)//4+1, col=(i-1)%4+1)

    # Adjust subplot titles font size and position
    for i in fig['layout']['annotations']:
        i['font'] = dict(size=10)
        i['y'] = i['y'] + 0.03  # Move subplot titles up slightly

    fig.show()

time: 1.21 ms (started: 2024-09-27 10:18:26 +00:00)


In [ ]:
####### DATA LOADING #######

file_name = '/content/AMZN_data.csv'            # File name
normalise = True               # Whether to normalise or not

# Load the data
stock_df = pd.read_csv(file_name)
stock_df = stock_df[['high', 'low', 'open', 'close', 'volume']]
stock_df['volume'] = stock_df['volume'].astype('float64')

# Calculate technical indicators
stock_df = calculate_technical_indicators(stock_df, normalise)
# stock_df

time: 10.4 ms (started: 2024-09-27 10:18:26 +00:00)


In [ ]:
####### DATASET GENERATION #######

days_input = 90              # Number of previous data points to use as input
days_output = 1              # Number of days to predict
train_split = 0.7            # Percentage of data to use for training
normalise = False            # Whether to normalise or not
shuffle = True               # To Shuffle or not to Shuffle

# Preprocess the data
X_train, y_train, X_test, y_test, test_data = prepare_train_test_data(stock_df, days_input, days_output, train_split, normalise, shuffle)

# Convert the data to PyTorch tensors
X_train = torch.from_numpy(X_train).double()
y_train = torch.from_numpy(y_train).double()
X_test = torch.from_numpy(X_test).double()
y_test = torch.from_numpy(y_test).double()

# Printing the shapes
print(f"X_train shape : {X_train.shape}")
print(f"y_train shape : {y_train.shape}")
print(f"X_test shape : {X_test.shape}")
print(f"y_test shape : {y_test.shape}")

X_train shape : torch.Size([775, 90, 18])
y_train shape : torch.Size([775, 1])
X_test shape : torch.Size([281, 90, 18])
y_test shape : torch.Size([281, 1])
time: 64.8 ms (started: 2024-09-27 10:18:26 +00:00)


In [ ]:
####### DATASET PARAMS #######

# Split the dataset into training and validation sets
training_size = 0.8                         # Ratio of Training Data
validation_size = 0.2                       # Ratio of Validation Data
train_size = int(training_size * len(X_train))
val_size = int(training_size * len(X_train))

# Preparing the training, validation and testing sets
train_dataset = (X_train[:train_size, :, :], y_train[:train_size, :])
val_dataset = (X_train[train_size:, :, :], y_train[train_size:, :])
test_dataset = (X_test, y_test)

# Printing the shapes
print(f"training dataset shape : {X_train[:train_size, :, :].shape}")
print(f"validation dataset shape : {X_train[train_size:].shape}")
print(f"testing dataset shape : {X_test[1].shape}")

training dataset shape : torch.Size([620, 90, 18])
validation dataset shape : torch.Size([155, 90, 18])
testing dataset shape : torch.Size([90, 18])
time: 1.52 ms (started: 2024-09-27 10:18:26 +00:00)


In [ ]:
####### MODEL PARAMS #######

# Initialize the model parameters
input_size = X_train.shape[-1]                # Number of features in your dataset
hidden_conv_size = 512                         # Hidden features after convolution
hidden_mlp_size = 1024                       # Hidden features after MLP
hidden_lstm_size_1 = 1024                       # Hidden features for self-attention
hidden_lstm_size_2 = 1024                       # Hidden features for self-attention
lstm_layers = 7                               # Number of LSTM\GRU layers
output_size = 1                               # Output size is 1 for single-value regression
kernel_sizes = np.array([2, 3, 5, 7, 14, 21])# Kernel sizes for the convolutional layers
seq_len = days_input                          # Sequence length for the LSTM
num_days = days_output                        # Number of days predicted
dropout_rate = 0.4                            # Dropout rate
num_heads = 16                                 # Number of attention heads

# Check if the hidden_lstm_size is divisible by num_heads
assert hidden_lstm_size_2 % num_heads == 0, "hidden_lstm_size must be divisible by num_heads"

# Initialize the traning hyperparameters
epochs = 20                                      # Number of epochs
batch_size = 64                              # Batch size
learning_rate = 0.001                           # Learning rate
l2_lambda = 0.0001                              # L2 regularisationn
save_path = '/content/model.pth'                         # Path to save the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

time: 702 µs (started: 2024-09-27 10:18:26 +00:00)


In [ ]:
# Add this at the end of the script
folder_path = '/content/'  # Replace with the actual path to your CSV folder
results = process_all_files(folder_path)

*********************************     Processing /content/AMZN_data.csv...     *********************************
Training new model for /content/AMZN_data.csv...


Epoch 1: 100%|██████████| 9/9 [00:11<00:00,  1.30s/batch, loss=33]


Validating...
Epoch 1, Training Loss: 33.02957380480237, Validation Loss: 0.9264729022979736

Test Loss: 0.9264729022979736
Best model saved with validation loss: 0.9265



Epoch 2: 100%|██████████| 9/9 [00:12<00:00,  1.34s/batch, loss=0.656]


Validating...
Epoch 2, Training Loss: 0.6557608313030667, Validation Loss: 0.5084091126918793

Test Loss: 0.5084091126918793
Best model saved with validation loss: 0.5084



...

Validating...
Epoch 18, Training Loss: 0.26099471085601383, Validation Loss: 0.271465465426445

Test Loss: 0.271465465426445


Epoch 19: 100%|██████████| 9/9 [00:11<00:00,  1.28s/batch, loss=0.252]


Validating...
Epoch 19, Training Loss: 0.2517640110519197, Validation Loss: 0.2706739231944084

Test Loss: 0.2706739231944084


Epoch 20: 100%|██████████| 9/9 [00:11<00:00,  1.28s/batch, loss=0.253]


Validating...
Epoch 20, Training Loss: 0.2530730929639604, Validation Loss: 0.27109578251838684

Test Loss: 0.27109578251838684
time: 4min 31s (started: 2024-09-27 10:18:26 +00:00)


In [ ]:
plot_results(results, epochs)

time: 43.6 ms (started: 2024-09-27 10:22:58 +00:00)


time: 50.6 ms (started: 2024-09-27 10:22:58 +00:00)


In [ ]:
####### MODEL INITIALISATION #######

# Initialising the model
model = CNNLSTMWithSelfAttentionLNN(input_size, hidden_conv_size, hidden_mlp_size, hidden_lstm_size_1, hidden_lstm_size_2, lstm_layers, output_size, kernel_sizes, seq_len, num_days, batch_size, device, l2_lambda, dropout_rate, num_heads)

# Defining the optimiser
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Defining the LR scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, verbose=True)

# Printing the model
print(f"**************** BACKBONE OF THE MODEL **************** \n\n {model}")

**************** BACKBONE OF THE MODEL **************** 

 CNNLSTMWithSelfAttentionLNN(
  (conv_blocks): ConvBlock(
    (conv_layers): ModuleList(
      (0): Conv1d(18, 512, kernel_size=(2,), stride=(1,), padding=same)
      (1): Conv1d(18, 512, kernel_size=(3,), stride=(1,), padding=same)
      (2): Conv1d(18, 512, kernel_size=(5,), stride=(1,), padding=same)
      (3): Conv1d(18, 512, kernel_size=(7,), stride=(1,), padding=same)
      (4): Conv1d(18, 512, kernel_size=(14,), stride=(1,), padding=same)
      (5): Conv1d(18, 512, kernel_size=(21,), stride=(1,), padding=same)
    )
    (bn_layers): ModuleList(
      (0-5): 6 x BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (pos_encoder): PositionalEncodingCNN(
      (dropout): Dropout(p=0.4, inplace=False)
    )
    (mlp): Conv1d(3073, 1024, kernel_size=(1,), stride=(1,))
    (bn_final): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation): ReLU()
 

In [ ]:
# # Load the model if saved
# if os.path.isfile(save_path):
#   print("Saved model loaded...\n")
#   model.load_state_dict(torch.load(save_path))

# # Train the model
# train_loss, val_loss = train(model, train_dataset, val_dataset, optimizer, scheduler, epochs=epochs, batch_size=batch_size, device=device, save_path=save_path)

time: 303 µs (started: 2024-09-27 10:22:59 +00:00)


In [ ]:
# # Create the plot
# fig = go.Figure()

# # Add training loss trace
# fig.add_trace(go.Scatter(
#     x=np.arange(len(train_loss)),
#     y=train_loss,
#     mode='lines+markers',
#     name='Training Loss'
# ))

# # Add validation loss trace
# fig.add_trace(go.Scatter(
#     x=np.arange(len(val_loss)),
#     y=val_loss,
#     mode='lines+markers',
#     name='Validation Loss'
# ))

# # Update layout
# fig.update_layout(
#     title='Training and Validation Loss Over Epochs',
#     xaxis_title='Epoch',
#     yaxis_title='Loss',
#     legend_title='Loss Type',
#     template='plotly_dark'
# )

# # Show the plot
# fig.show()

time: 279 µs (started: 2024-09-27 10:22:59 +00:00)


In [ ]:

# Load the model if saved
if os.path.isfile(save_path):
  print("Saved model loaded...\n\n")
  model.load_state_dict(torch.load(save_path))

# # Reset hidden states before testing
# model.reset_hidden()

# Test the model
test_loss, outputs, labels = test(model, test_dataset, batch_size, device)

Saved model loaded...


Test Loss: 4.413557648658752
time: 2.14 s (started: 2024-09-27 10:22:59 +00:00)


In [ ]:
print(outputs.shape)
print(labels.shape)

torch.Size([64, 1])
torch.Size([64, 1])
time: 533 µs (started: 2024-09-27 10:23:01 +00:00)


In [ ]:
print(outputs)

tensor([[-0.8070],
        [-0.8074],
        [-0.8072],
        [-0.8071],
        [-0.8072],
        [-0.8069],
        [-0.8071],
        [-0.8071],
        [-0.8073],
        [-0.8072],
        [-0.8072],
        [-0.8075],
        [-0.8074],
        [-0.8072],
        [-0.8072],
        [-0.8070],
        [-0.8072],
        [-0.8071],
        [-0.8070],
        [-0.8072],
        [-0.8071],
        [-0.8072],
        [-0.8071],
        [-0.8072],
        [-0.8071],
        [-0.8072],
        [-0.8071],
        [-0.8072],
        [-0.8071],
        [-0.8072],
        [-0.8073],
        [-0.8074],
        [-0.8073],
        [-0.8073],
        [-0.8072],
        [-0.8073],
        [-0.8072],
        [-0.8072],
        [-0.8073],
        [-0.8072],
        [-0.8073],
        [-0.8072],
        [-0.8074],
        [-0.8071],
        [-0.8073],
        [-0.8071],
        [-0.8073],
        [-0.8072],
        [-0.8072],
        [-0.8072],
        [-0.8074],
        [-0.8072],
        [-0.

In [ ]:
print(labels)

tensor([[1.3538],
        [1.3464],
        [1.3760],
        [1.4305],
        [1.4615],
        [1.4665],
        [1.4530],
        [1.4807],
        [1.5017],
        [1.5088],
        [1.5208],
        [1.5307],
        [1.4877],
        [1.4509],
        [1.4378],
        [1.3790],
        [1.4130],
        [1.4024],
        [1.4007],
        [1.8558],
        [1.8909],
        [1.8712],
        [1.8655],
        [1.8320],
        [1.8936],
        [1.9256],
        [1.9345],
        [1.9689],
        [1.9556],
        [1.9423],
        [1.9558],
        [1.9829],
        [1.9470],
        [1.9845],
        [1.9583],
        [1.9457],
        [1.9923],
        [2.0514],
        [2.1570],
        [2.1918],
        [2.1839],
        [2.0695],
        [2.1243],
        [2.0733],
        [1.9727],
        [1.9997],
        [2.0379],
        [2.0642],
        [2.0720],
        [2.0965],
        [2.0829],
        [2.0796],
        [2.1155],
        [2.1327],
        [2.1732],
        [2

In [ ]:
# Convert tensors to numpy arrays
outputs_np = outputs.cpu().numpy()
labels_np = labels.cpu().numpy()

# Extract the entire arrays (no need to get the first value as the size is already [929, 1])
outputs_np = outputs_np.flatten()
labels_np = labels_np.flatten()

print(outputs_np.shape)
print(labels_np.shape)
print("\n\n")

# Plot using Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(labels_np)), y=labels_np, mode='lines', name='Label'))
fig.add_trace(go.Scatter(x=np.arange(len(outputs_np)), y=outputs_np, mode='lines', name='Output'))

fig.update_layout(
    title='Labels and Outputs',
    xaxis_title='Index',
    yaxis_title='Value',
    legend_title='Legend'
)

fig.show()


(64,)
(64,)





time: 12.9 ms (started: 2024-09-27 10:23:01 +00:00)


In [ ]:
array_output = assign_trend_sign(outputs_np)
array_labels = assign_trend_sign(labels_np)

matching_val = count_matching_elements(array_output, array_labels)
print(f"Matching elements precentage: {matching_val}")

Matching elements precentage: 46.875
time: 706 µs (started: 2024-09-27 10:23:01 +00:00)


In [ ]:
print(array_output)

[ 0. -1.  1.  1. -1.  1. -1. -1. -1.  1. -1. -1.  1.  1.  1.  1. -1.  1.
  1. -1.  1. -1.  1. -1.  1. -1.  1. -1.  1. -1. -1. -1.  1. -1.  1. -1.
  1.  1. -1.  1. -1.  1. -1.  1. -1.  1. -1.  1. -1.  1. -1.  1. -1.  1.
 -1.  1. -1.  1. -1.  1. -1.  1. -1.  1.]
time: 845 µs (started: 2024-09-27 10:23:01 +00:00)


In [ ]:
print(array_labels)

[ 0. -1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1. -1. -1. -1. -1.  1. -1.
 -1.  1.  1. -1. -1. -1.  1.  1.  1.  1. -1. -1.  1.  1. -1.  1. -1. -1.
  1.  1.  1.  1. -1. -1.  1. -1. -1.  1.  1.  1.  1.  1. -1. -1.  1.  1.
  1. -1. -1. -1. -1.  1.  1.  1. -1.  1.]
time: 896 µs (started: 2024-09-27 10:23:01 +00:00)


In [ ]:
# Plot using Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(array_labels)), y=array_labels, mode='lines', name='Label'))
fig.add_trace(go.Scatter(x=np.arange(len(array_output)), y=array_output, mode='lines', name='Output'))

fig.update_layout(
    title='Labels and Outputs',
    xaxis_title='Index',
    yaxis_title='Value',
    legend_title='Legend'
)

fig.show()

time: 14.3 ms (started: 2024-09-27 10:23:01 +00:00)


In [ ]:
# Assuming array_output and array_labels are already defined and are 1D numpy arrays
# containing the trend signs (-1, 0, 1) for each element in the sequence
#. t\o  -1  0  1
#.  -1
#.   0
#.   1

# Compute the confusion matrix
conf_matrix = confusion_matrix(array_labels, array_output)

print(f"Confusion Matrix: \n{conf_matrix}")

Confusion Matrix: 
[[12  0 15]
 [ 0  1  0]
 [19  0 17]]
time: 3.35 ms (started: 2024-09-27 10:23:01 +00:00)


In [ ]:
array_output = assign_trend_sign(np.roll(outputs_np, -1))
array_labels = assign_trend_sign(labels_np)

matching_val = count_matching_elements(array_output, array_labels)
print(f"Matching elements precentage if output is shifted: {matching_val}")

Matching elements precentage if output is shifted: 51.5625
time: 1.37 ms (started: 2024-09-27 10:23:01 +00:00)


In [ ]:
# Calculate Sharpe ratio
returns = pd.Series(outputs_np).pct_change().dropna()
sharpe_ratio = calculate_sharpe_ratio(returns)
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")

Sharpe Ratio: -1525.10
time: 2.38 ms (started: 2024-09-27 10:23:01 +00:00)


In [ ]:
def trading_strategy(labels, output, initial_balance=10000, share_price=100):
    balance = initial_balance  # Starting with some initial balance
    shares = 0  # No shares initially
    total_profit = 0
    spent=0
    total_spent=0;
    for i in range(len(output)):

        if output[i] > 0:  # Predicted profit day
            # Buy as many shares as possible
            shares_to_buy = 10
            shares += shares_to_buy
            balance -= shares_to_buy * share_price
            spent=10*share_price
            print(f"Day {i}: Bought {shares_to_buy} shares at {share_price} each, Balance: {balance}, Shares: {shares}")

        elif output[i] < 0:  # Predicted loss day
            # Sell all shares
            balance += shares * share_price
            profit = shares * (share_price)-spent
            total_spent+= spent

            spent=0
            total_profit += profit
            print(f"Day {i}: Sold {shares} shares at {share_price} each, Balance: {balance}, Profit: {profit}, Total Profit: {total_profit}")
            shares = 0  # Reset shares after selling
        share_price+=labels[i]
    # At the end, sell any remaining shares
    print(f"Total Spent {total_spent}")
    if shares > 0:
        balance = shares * share_price
        print(f"Final Sale: Sold remaining {shares} shares, Final Balance: {balance}")

    print(f"Final Total Profit: {total_profit}")
    return total_profit, balance


trading_strategy(array_labels, array_output)

Day 1: Bought 10 shares at 100.0 each, Balance: 9000.0, Shares: 10
Day 2: Bought 10 shares at 99.0 each, Balance: 8010.0, Shares: 20
Day 3: Sold 20 shares at 100.0 each, Balance: 10010.0, Profit: 1010.0, Total Profit: 1010.0
Day 4: Bought 10 shares at 101.0 each, Balance: 9000.0, Shares: 10
Day 5: Sold 10 shares at 102.0 each, Balance: 10020.0, Profit: 10.0, Total Profit: 1020.0
Day 6: Sold 0 shares at 103.0 each, Balance: 10020.0, Profit: 0.0, Total Profit: 1020.0
Day 7: Sold 0 shares at 102.0 each, Balance: 10020.0, Profit: 0.0, Total Profit: 1020.0
Day 8: Bought 10 shares at 103.0 each, Balance: 8990.0, Shares: 10
Day 9: Sold 10 shares at 104.0 each, Balance: 10030.0, Profit: 10.0, Total Profit: 1030.0
Day 10: Sold 0 shares at 105.0 each, Balance: 10030.0, Profit: 0.0, Total Profit: 1030.0
Day 11: Bought 10 shares at 106.0 each, Balance: 8970.0, Shares: 10
Day 12: Bought 10 shares at 107.0 each, Balance: 7900.0, Shares: 20
Day 13: Bought 10 shares at 106.0 each, Balance: 6840.0, Sha

(6260.0, 2180.0)

time: 26.9 ms (started: 2024-09-27 10:23:01 +00:00)


time: 33.7 ms (started: 2024-09-27 10:23:01 +00:00)


In [ ]:
def trading_strategy(labels_np, outputs_np, initial_balance=10000):
    balance = initial_balance
    shares = 0
    total_profit = 0
    total_spent = 0
    share_price = initial_share_price
    last_buy_price = 0
    last_sell_price = 0
    initial_share_price = labels_np[0]

    for i in range(1, len(outputs_np)):  # Start from 1 to compare with previous day
        # Calculate the delta (change) between current and previous day's predictions
        delta = outputs_np[i] - outputs_np[i-1]

        # Use actual price from labels for calculations
        current_price = share_price + labels_np[i] - labels_np[i-1]

        if delta > 0 and balance > 0:  # Buy signal: prediction increased
            price_change = (current_price - last_sell_price) / last_sell_price if last_sell_price > 0 else 0
            if price_change >= 0:#0.005:  # Only buy if there's a 5% increase from last sell
                # Buy as many shares as possible
                shares_to_buy = balance // current_price
                if shares_to_buy > 0:
                    shares += shares_to_buy
                    cost = shares_to_buy * current_price
                    balance -= cost
                    total_spent += cost
                    last_buy_price = current_price
                    print(f"Day {i+1}: Bought {shares_to_buy} shares at {current_price:.2f} each, Balance: {balance:.2f}, Shares: {shares}")

        elif delta < 0 and shares > 0:  # Sell signal: prediction decreased
            price_change = (last_buy_price - current_price) / last_buy_price if last_buy_price > 0 else 0
            if price_change >= 0:#0.005:  # Only sell if there's a 5% decrease from last buy
                # Sell all shares
                sale_amount = shares * current_price
                profit = sale_amount - total_spent
                tax = profit * 0.1  # 10% tax on profit
                profit_after_tax = profit - tax
                balance += sale_amount - tax
                total_profit += profit_after_tax
                last_sell_price = current_price
                print(f"Day {i+1}: Sold {shares} shares at {current_price:.2f} each, Balance: {balance:.2f}, Profit: {profit_after_tax:.2f}, Tax: {tax:.2f}, Total Profit: {total_profit:.2f}")
                shares = 0
                total_spent = 0

        # Update share price for the next day
        share_price = current_price

    # Sell any remaining shares at the end
    if shares > 0:
        final_sale_amount = shares * share_price
        final_profit = final_sale_amount - total_spent
        final_tax = final_profit * 0.1
        final_profit_after_tax = final_profit - final_tax
        total_profit += final_profit_after_tax
        balance += final_sale_amount - final_tax
        print(f"Final Sale: Sold remaining {shares} shares at {share_price:.2f}, Final Balance: {balance:.2f}, Final Profit: {final_profit_after_tax:.2f}, Tax: {final_tax:.2f}")

    profit_percentage = (total_profit / initial_balance) * 100
    print(f"Final Total Profit: {total_profit:.2f}")
    print(f"Profit Percentage: {profit_percentage:.2f}%")

    return total_profit, balance, profit_percentage

# Example usage:
trading_strategy(labels_np, outputs_np)

Day 3: Bought 9783.0 shares at 1.02 each, Balance: 0.13, Shares: 9783.0
Final Sale: Sold remaining 9783.0 shares at 1.81, Final Balance: 16970.47, Final Profit: 6970.47, Tax: 774.50
Final Total Profit: 6970.47
Profit Percentage: 69.70%


(6970.472167897225, 16970.472167897224, 69.70472167897225)

time: 8.39 ms (started: 2024-09-27 10:26:22 +00:00)


In [ ]:
outputs_np

array([-0.8070068 , -0.80736333, -0.807152  , -0.8071165 , -0.8071647 ,
       -0.806917  , -0.80705535, -0.80705816, -0.8072763 , -0.80715287,
       -0.8071756 , -0.80747586, -0.8073661 , -0.8071885 , -0.8071511 ,
       -0.8069561 , -0.80718285, -0.8071421 , -0.80695456, -0.8072178 ,
       -0.8071367 , -0.8071509 , -0.8071334 , -0.8071653 , -0.80714035,
       -0.8071895 , -0.8071322 , -0.8072094 , -0.8071477 , -0.8072458 ,
       -0.8072735 , -0.80735546, -0.80725455, -0.8072779 , -0.8072365 ,
       -0.8072556 , -0.80723757, -0.80719924, -0.8072916 , -0.8071953 ,
       -0.8072558 , -0.8071613 , -0.8074364 , -0.8071415 , -0.8073312 ,
       -0.80713564, -0.80727273, -0.80718505, -0.8072415 , -0.80722976,
       -0.80744845, -0.8072293 , -0.8074607 , -0.8071113 , -0.8071812 ,
       -0.8070144 , -0.8072222 , -0.80701786, -0.80719584, -0.80694824,
       -0.80740124, -0.8072233 , -0.80723864, -0.80707043], dtype=float32)

time: 3.66 ms (started: 2024-09-27 10:23:01 +00:00)


time: 8.15 ms (started: 2024-09-27 10:23:01 +00:00)
